In [1]:
import sys

import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as agg
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [2]:
df = hd.read_analysis_data()
hd.inspect(df)

Time for read_analysis_data            : 2.05 seconds
(85,364, 26)


,user_id,month,date,active_accounts,age,annual_income,entropy,entropyz,female,has_sa_inflows,month_income,month_spend,sa_inflows,spend_communication,spend_finance,spend_hobbies,spend_household,spend_motor,spend_other_spend,spend_retail,spend_services,spend_travel,txn_count_ca,txn_count_sa,txns_count,txns_value
124,37,2,2012-02-29,"[287407, 287405, 287406, 287404]",27.0,18768.632812,2.899397,1.137196,0.0,1,1687.530029,528.239990,3340.0,50.0,0.0,0.0,312.640015,0.0,46.799999,0.0,118.800003,0.0,8.0,3.0,22,9495.810547
125,37,3,2012-03-31,"[287405, 287407, 287406, 287404]",27.0,18768.632812,3.039149,1.661248,0.0,0,1085.790039,429.880005,0.0,50.0,0.0,0.0,327.500000,0.0,40.000000,0.0,12.380000,0.0,9.0,2.0,16,2147.699951


ISA transfers

In [5]:
dft = hd.read_txn_data('777')
hd.inspect(dft)

Time for read_txn_data                 : 2.73 seconds
(651,789, 18)


,date,user_id,amount,desc,merchant,tag_group,tag,account_id,account_last_refreshed,account_provider,account_type,debit,female,id,latest_balance,postcode,tag_auto,yob
0,2012-02-01,777,400.00,<mdbremoved> - s/o,NaN,transfers,other_transfers,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688300,364.220001,wa1 4,transfers,1969.0
1,2012-02-01,777,3.03,aviva pa - d/d,aviva,spend,finance,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688299,364.220001,wa1 4,health insurance,1969.0


In [6]:
dft[dft.desc.str.contains('atm')]

,date,user_id,amount,desc,merchant,tag_group,tag,account_id,account_last_refreshed,account_provider,account_type,debit,female,id,latest_balance,postcode,tag_auto,yob
5,2012-02-06,777,40.0,natwest 05feb - atm,NaN,spend,other_spend,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688304,364.220001,wa1 4,cash,1969.0
28,2012-04-23,777,60.0,santander 20apr - atm,NaN,spend,other_spend,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688340,364.220001,wa1 4,cash,1969.0
39,2012-04-30,777,40.0,royal bank 30apr - atm,NaN,spend,other_spend,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688354,364.220001,wa1 4,cash,1969.0
50,2012-05-08,777,40.0,natwest 05may - atm,NaN,spend,other_spend,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688412,364.220001,wa1 4,cash,1969.0
54,2012-05-09,777,40.0,natwest 09may - atm,NaN,spend,other_spend,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688415,364.220001,wa1 4,cash,1969.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649892,2019-07-19,587777,200.0,<mdbremoved> acton 1 atm,NaN,spend,other_spend,1711781,2020-08-16 18:36:00,barclays,current,True,0.0,790871888,5699.700195,w7 3,cash,1973.0
650212,2019-09-16,587777,150.0,sainsburys bank sainsburys bank 16sep 19.17 atm,NaN,spend,other_spend,1711781,2020-08-16 18:36:00,barclays,current,True,0.0,790871821,5699.700195,w7 3,cash,1973.0
650309,2019-10-04,587777,100.0,sainsburys bank sainsburys bank 04oct 15.28 atm,NaN,spend,other_spend,1711781,2020-08-16 18:36:00,barclays,current,True,0.0,790871798,5699.700195,w7 3,cash,1973.0
650629,2019-12-01,587777,70.0,notemachine costcutter boston 01dec 14.09 atm,NaN,spend,other_spend,1711781,2020-08-16 18:36:00,barclays,current,True,0.0,790871741,5699.700195,w7 3,cash,1973.0
